In [18]:
%matplotlib inline
%env KERAS_BACKEND=tensorflow

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import common

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.losses import mean_squared_error
from keras.losses import mean_absolute_percentage_error
from keras.callbacks import TensorBoard

from sklearn import preprocessing
from sklearn import metrics

env: KERAS_BACKEND=tensorflow


raining data = mid June-mid July, test data = mid of Juy to Aug
 
1)      Tonnage prediction
Y = loadsys
Features = drybulb, relative humidity (rh), Time Stamp (weekdays/weekends), hour
 
2)      Chiller plant
Y = systotpow
Features = loadsys, Time Stamp (weekdays/weekends), hour, dry bulb, and relative humidity (rh)
 
 

In [23]:
import importlib
importlib.reload(common)

<module 'common' from '/usa/suryak/Notebooks/chiller-plants/notebooks/common.py'>

In [24]:
df = common.load_df("../data/insead", "*.csv")
print(common.load_df.cache_info())

# process. at last, normalize.
_cols = ["drybulb", "loadsys", "rh", "systotpower"]
df = common.Process.replace_nulls(df, cols=_cols)
df = common.Process.replace_with_zero(df, cols=_cols)
df = common.Process.set_ctwk_total(df, cols=_cols)
df = common.Process.get_normalized_df(df, scale=(0.1, 1), cols=_cols)

# add few more features.
df["weekday"] = df.index.weekday
df["weekend"] = df.weekday.apply(lambda x: int(x in (5,6)))
df["hour"] = df.index.hour

CacheInfo(hits=0, misses=1, maxsize=None, currsize=1)


## Experiment 1
```
X = DB, Rh, Hour, Weekday
y = Loadsys

Train: Mid Jun-Jul
Test: Mid Jul-Aug
```

In [26]:
# manually select training and validaiton data.
batch_size = 200
train_df = df["2016-06-15":"2016-07-14"]
validation_df = df["2016-07-15":"2016-08-14"]

# The recipe.
features = ["drybulb", "rh", "hour", "weekend"]
target = ["loadsys"]
lookback = 5

# Prepare features and target vectors
train_x, train_y = common.prepare_keras_data(train_df, features, target, lookback, batch_size)
validation_x, validation_y = common.prepare_keras_data(validation_df, features, target, lookback, batch_size)
print(train_x.shape, train_y.shape)

(43000, 1, 20) (43000, 1, 1)


In [27]:
model = Sequential([
    LSTM(20, batch_input_shape=(batch_size, train_x.shape[1], train_x.shape[2]), stateful=True, return_sequences=True),
    LSTM(40, stateful=True, return_sequences=True),
    LSTM(40, stateful=True, return_sequences=True),
    LSTM(10, stateful=True, return_sequences=True),
    Dense(1)
])

model.compile(
    loss=mean_squared_error,
    optimizer="adam",
    metrics=["accuracy", mean_absolute_percentage_error]
)

In [ ]:
for epoch in range(150):
    history = model.fit(
        x=train_x,
        y=train_y,
        batch_size=batch_size,
        verbose=0,
        shuffle=False,
        epochs=1,
        validation_data=(validation_x, validation_y),
        callbacks=[TensorBoard(log_dir="../output/keras1")])
    model.reset_states()
    if epoch % 50 == 0:
        print("At epoch ", epoch)

At epoch  0
